In [1]:
#%%
from ns3gym import ns3env
from comet_ml import Experiment, Optimizer
import tqdm
import subprocess
from collections import deque
import numpy as np

from agents.ddpg.agent import Agent, Config
from agents.ddpg.model import Actor
from agents.teacher import Teacher, EnvWrapper
from preprocessor import Preprocessor


In [2]:
#%%
scenario = "convergence"

simTime = 60 # seconds
stepTime = 0.01  # seconds
history_length = 300

EPISODE_COUNT = 15
steps_per_ep = int(simTime/stepTime)

sim_args = {
    "simTime": simTime,
    "envStepTime": stepTime,
    "historyLength": history_length,
    "agentType": Agent.TYPE,
    "scenario": "convergence",
    "nWifi": 30,
}

print("Steps per episode:", steps_per_ep)

threads_no = 1
env = EnvWrapper(threads_no, **sim_args)

#%%
env.reset()
ob_space = env.observation_space
ac_space = env.action_space

print("Observation space shape:", ob_space)
print("Action space shape:", ac_space)

assert ob_space is not None

Steps per episode: 6000
Waiting for simulation script to connect on port: tcp://localhost:21767
Please start proper ns-3 simulation script using ./waf --run "..."
Waf: Entering directory `/home/samuel/ns-allinone-3.29/ns-3.29/build'
Waf: Leaving directory `/home/samuel/ns-allinone-3.29/ns-3.29/build'
Build commands will be stored in build/compile_commands.json
'build' finished successfully (24.422s)


Ns3Env parameters:
--nWifi: 30
--simulationTime: 60
--openGymPort: 21767
--envStepTime: 0.01
--seed: -1
--agentType: continuous
--scenario: convergence
--dryRun: 0
Simulation started


Observation space shape: (1, 300)
Action space shape: (1, 1)


Simulation process id: 3240 (parent (waf shell) id: 3147)
Waiting for Python process to connect on port: tcp://localhost:21767
Please start proper Python Gym Agent


In [3]:
#%%
teacher = Teacher(env, 1, Preprocessor(False))

lr_actor = 4e-4
lr_critic = 4e-3

config = Config(buffer_size=4*steps_per_ep*threads_no, batch_size=32, gamma=0.7, tau=1e-3, lr_actor=lr_actor, lr_critic=lr_critic, update_every=1)
agent = Agent(state_size=30, action_size=1, config=config, actor_layers=[8, 128, 16], critic_layers=[8,128,16])

# Test the model
hyperparams = {**config.__dict__, **sim_args}
tags = ["Rew: normalized speed",
        f"{Agent.NAME}",
        sim_args['scenario'],
        f"Actor: {lr_actor}",
        f"Critic: {lr_critic}",
        f"Instances: {threads_no}",
        f"Station count: {sim_args['nWifi']}",
        *[f"{key}: {sim_args[key]}" for key in list(sim_args)[:3]]]
# agent.save()
logger = teacher.train(agent, EPISODE_COUNT,
                        simTime=simTime,
                        stepTime=stepTime,
                        history_length=history_length,
                        send_logs=True,
                        experimental=True,
                        tags=tags,
                        parameters=hyperparams)
# logger = teacher.eval(agent,
#                         simTime=simTime,
#                         stepTime=stepTime,
#                         history_length=history_length,
#                         tags=tags,
#                         parameters=hyperparams)
# agent.save()

CuDNN version: 7102
cpu


COMET INFO: Experiment is live on comet.ml https://www.comet.com/sheila-janota/rl-in-wifi/3b28d08e559f4dedbcc4d87c9973ba2c

  0%|          | 0/6300 [00:00<?, ?it/s]

0


  5%|▌         | 333/6300 [02:02<45:23,  2.19it/s, curr_speed=34.63 Mbps, mb_sent=11.08 Mb]

------- STARTED TRAINING -------


...

Sent 2214.53 Mb/s.	Mean speed: 36.91 Mb/s	Episode 14/15 finished

14
Turning off noise


100%|██████████| 6300/6300 [44:09<00:00,  2.38it/s, curr_speed=34.10 Mbps, mb_sent=2230.85 Mb]


Waiting for simulation script to connect on port: tcp://localhost:41119
Please start proper ns-3 simulation script using ./waf --run "..."
Waf: Entering directory `/home/samuel/ns-allinone-3.29/ns-3.29/build'
Waf: Leaving directory `/home/samuel/ns-allinone-3.29/ns-3.29/build'
Build commands will be stored in build/compile_commands.json
'build' finished successfully (2.034s)


Ns3Env parameters:
--nWifi: 30
--simulationTime: 60
--openGymPort: 41119
--envStepTime: 0.01
--seed: -1
--agentType: continuous
--scenario: convergence
--dryRun: 0
Simulation started
Simulation process id: 805535 (parent (waf shell) id: 805504)
Waiting for Python process to connect on port: tcp://localhost:41119
Please start proper Python Gym Agent
COMET INFO: ----------------------------
COMET INFO: Comet.ml Experiment Summary:
COMET INFO:   Data:
COMET INFO:     url: https://www.comet.com/sheila-janota/rl-in-wifi/3b28d08e559f4dedbcc4d87c9973ba2c
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     Chosen CW [89985]           : (16.0, 1024.0)
COMET INFO:     Cumulative reward [15]      : (2520.30078125, 3082.8232421875)
COMET INFO:     Current throughput [89985]  : (25.689701080322266, 59.78997039794922)
COMET INFO:     Fairness index [89985]      : (0.039413999766111374, 0.997402012348175)
COMET INFO:     Megabytes sent [89985]      : (0.2568970024585724, 2230.8485365509987)
COM

Sent 2230.85 Mb/s.	Mean speed: 37.18 Mb/s	Episode 15/15 finished

Training finished.


# Execution num2:

In [1]:
#%%
from ns3gym import ns3env
from comet_ml import Experiment, Optimizer
import tqdm
import subprocess
from collections import deque
import numpy as np

from agents.ddpg.agent import Agent, Config
from agents.ddpg.model import Actor
from agents.teacher import Teacher, EnvWrapper
from preprocessor import Preprocessor

In [2]:
#%%
scenario = "convergence"

simTime = 60 # seconds
stepTime = 0.01  # seconds
history_length = 300

EPISODE_COUNT = 15
steps_per_ep = int(simTime/stepTime)

sim_args = {
    "simTime": simTime,
    "envStepTime": stepTime,
    "historyLength": history_length,
    "agentType": Agent.TYPE,
    "scenario": "convergence",
    "nWifi": 30,
}

print("Steps per episode:", steps_per_ep)

threads_no = 1
env = EnvWrapper(threads_no, **sim_args)

#%%
env.reset()
ob_space = env.observation_space
ac_space = env.action_space

print("Observation space shape:", ob_space)
print("Action space shape:", ac_space)

assert ob_space is not None

Steps per episode: 6000
Waiting for simulation script to connect on port: tcp://localhost:28932
Please start proper ns-3 simulation script using ./waf --run "..."
Waf: Entering directory `/home/samuel/ns-allinone-3.29/ns-3.29/build'
Waf: Leaving directory `/home/samuel/ns-allinone-3.29/ns-3.29/build'
Build commands will be stored in build/compile_commands.json
'build' finished successfully (2.009s)


Ns3Env parameters:
--nWifi: 30
--simulationTime: 60
--openGymPort: 28932
--envStepTime: 0.01
--seed: -1
--agentType: continuous
--scenario: convergence
--dryRun: 0
Simulation started


Observation space shape: (1, 300)
Action space shape: (1, 1)


Simulation process id: 3635 (parent (waf shell) id: 3614)
Waiting for Python process to connect on port: tcp://localhost:28932
Please start proper Python Gym Agent


In [3]:
#%%
teacher = Teacher(env, 1, Preprocessor(False))

lr_actor = 4e-4
lr_critic = 4e-3

config = Config(buffer_size=4*steps_per_ep*threads_no, batch_size=32, gamma=0.7, tau=1e-3, lr_actor=lr_actor, lr_critic=lr_critic, update_every=1)
agent = Agent(history_length, action_size=1, config=config, actor_layers=[8, 128, 16], critic_layers=[8,128,16])

# Test the model
hyperparams = {**config.__dict__, **sim_args}
tags = ["Rew: normalized speed",
        f"{Agent.NAME}",
        sim_args['scenario'],
        f"Actor: {lr_actor}",
        f"Critic: {lr_critic}",
        f"Instances: {threads_no}",
        f"Station count: {sim_args['nWifi']}",
        *[f"{key}: {sim_args[key]}" for key in list(sim_args)[:3]]]
# agent.save()
logger = teacher.train(agent, EPISODE_COUNT,
                        simTime=simTime,
                        stepTime=stepTime,
                        history_length=history_length,
                        send_logs=True,
                        experimental=True,
                        tags=tags,
                        parameters=hyperparams)
# logger = teacher.eval(agent,
#                         simTime=simTime,
#                         stepTime=stepTime,
#                         history_length=history_length,
#                         tags=tags,
#                         parameters=hyperparams)
# agent.save()

CuDNN version: 7102
cpu


COMET INFO: Experiment is live on comet.ml https://www.comet.com/sheila-janota/rl-in-wifi/1dfc05f17e9c43798b071fdf605a54a1

  0%|          | 1/6300 [00:00<19:18,  5.44it/s, curr_speed=0.00 Mbps, mb_sent=0.00 Mb]

0


  5%|▌         | 334/6300 [01:05<18:46,  5.30it/s, curr_speed=31.24 Mbps, mb_sent=10.31 Mb]

------- STARTED TRAINING -------


...

Sent 2255.14 Mb/s.	Mean speed: 37.59 Mb/s	Episode 14/15 finished

14
Turning off noise


100%|██████████| 6300/6300 [40:56<00:00,  2.56it/s, curr_speed=36.33 Mbps, mb_sent=2254.62 Mb]


Waiting for simulation script to connect on port: tcp://localhost:18545
Please start proper ns-3 simulation script using ./waf --run "..."
Waf: Entering directory `/home/samuel/ns-allinone-3.29/ns-3.29/build'
Waf: Leaving directory `/home/samuel/ns-allinone-3.29/ns-3.29/build'
Build commands will be stored in build/compile_commands.json
'build' finished successfully (2.013s)


Ns3Env parameters:
--nWifi: 30
--simulationTime: 60
--openGymPort: 18545
--envStepTime: 0.01
--seed: -1
--agentType: continuous
--scenario: convergence
--dryRun: 0
Simulation started
Simulation process id: 1006467 (parent (waf shell) id: 1006444)
Waiting for Python process to connect on port: tcp://localhost:18545
Please start proper Python Gym Agent
COMET INFO: ----------------------------
COMET INFO: Comet.ml Experiment Summary:
COMET INFO:   Data:
COMET INFO:     url: https://www.comet.com/sheila-janota/rl-in-wifi/1dfc05f17e9c43798b071fdf605a54a1
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     Chosen CW [89985]           : (16.0, 1024.0)
COMET INFO:     Cumulative reward [15]      : (2532.438232421875, 3141.693603515625)
COMET INFO:     Current throughput [89985]  : (21.56817626953125, 61.74462890625)
COMET INFO:     Fairness index [89985]      : (0.03946699947118759, 0.9973530173301697)
COMET INFO:     Megabytes sent [89985]      : (0.3015750050544739, 2255.1420848071575)

Sent 2254.62 Mb/s.	Mean speed: 37.58 Mb/s	Episode 15/15 finished

Training finished.


# Execution num3:

In [1]:
#%%
from ns3gym import ns3env
from comet_ml import Experiment, Optimizer
import tqdm
import subprocess
from collections import deque
import numpy as np

from agents.ddpg.agent import Agent, Config
from agents.ddpg.model import Actor
from agents.teacher import Teacher, EnvWrapper
from preprocessor import Preprocessor

In [2]:
#%%
scenario = "convergence"

simTime = 60 # seconds
stepTime = 0.01  # seconds
history_length = 300

EPISODE_COUNT = 15
steps_per_ep = int(simTime/stepTime)

sim_args = {
    "simTime": simTime,
    "envStepTime": stepTime,
    "historyLength": history_length,
    "agentType": Agent.TYPE,
    "scenario": "convergence",
    "nWifi": 30,
}

print("Steps per episode:", steps_per_ep)

threads_no = 1
env = EnvWrapper(threads_no, **sim_args)

#%%
env.reset()
ob_space = env.observation_space
ac_space = env.action_space

print("Observation space shape:", ob_space)
print("Action space shape:", ac_space)

assert ob_space is not None

Steps per episode: 6000
Waiting for simulation script to connect on port: tcp://localhost:41958
Please start proper ns-3 simulation script using ./waf --run "..."
Waf: Entering directory `/home/samuel/ns-allinone-3.29/ns-3.29/build'
Waf: Leaving directory `/home/samuel/ns-allinone-3.29/ns-3.29/build'
Build commands will be stored in build/compile_commands.json
'build' finished successfully (1.968s)


Ns3Env parameters:
--nWifi: 30
--simulationTime: 60
--openGymPort: 41958
--envStepTime: 0.01
--seed: -1
--agentType: continuous
--scenario: convergence
--dryRun: 0
Simulation started


Observation space shape: (1, 300)
Action space shape: (1, 1)


Simulation process id: 3348 (parent (waf shell) id: 3331)
Waiting for Python process to connect on port: tcp://localhost:41958
Please start proper Python Gym Agent


In [3]:
#%%
teacher = Teacher(env, 1, Preprocessor(False))

lr_actor = 4e-4
lr_critic = 4e-3

config = Config(buffer_size=4*steps_per_ep*threads_no, batch_size=32, gamma=0.7, tau=1e-3, lr_actor=lr_actor, lr_critic=lr_critic, update_every=1)
agent = Agent(history_length, action_size=1, config=config, actor_layers=[8, 128, 16], critic_layers=[8,128,16])

# Test the model
hyperparams = {**config.__dict__, **sim_args}
tags = ["Rew: normalized speed",
        f"{Agent.NAME}",
        sim_args['scenario'],
        f"Actor: {lr_actor}",
        f"Critic: {lr_critic}",
        f"Instances: {threads_no}",
        f"Station count: {sim_args['nWifi']}",
        *[f"{key}: {sim_args[key]}" for key in list(sim_args)[:3]]]
# agent.save()
logger = teacher.train(agent, EPISODE_COUNT,
                        simTime=simTime,
                        stepTime=stepTime,
                        history_length=history_length,
                        send_logs=True,
                        experimental=True,
                        tags=tags,
                        parameters=hyperparams)
# logger = teacher.eval(agent,
#                         simTime=simTime,
#                         stepTime=stepTime,
#                         history_length=history_length,
#                         tags=tags,
#                         parameters=hyperparams)
# agent.save()

CuDNN version: 7102
cpu


COMET INFO: Experiment is live on comet.ml https://www.comet.com/sheila-janota/rl-in-wifi/d8358c0294004b81a21bd48d5e3027d7

  0%|          | 0/6300 [00:00<?, ?it/s]

0


  5%|▌         | 333/6300 [01:01<18:50,  5.28it/s, curr_speed=32.39 Mbps, mb_sent=10.37 Mb]

------- STARTED TRAINING -------


...

Waiting for simulation script to connect on port: tcp://localhost:28039
Please start proper ns-3 simulation script using ./waf --run "..."
Waf: Entering directory `/home/samuel/ns-allinone-3.29/ns-3.29/build'
Waf: Leaving directory `/home/samuel/ns-allinone-3.29/ns-3.29/build'
Build commands will be stored in build/compile_commands.json
'build' finished successfully (2.360s)


Ns3Env parameters:
--nWifi: 30
--simulationTime: 60
--openGymPort: 28039
--envStepTime: 0.01
--seed: -1
--agentType: continuous
--scenario: convergence
--dryRun: 0
Simulation started
Simulation process id: 762724 (parent (waf shell) id: 762684)
Waiting for Python process to connect on port: tcp://localhost:28039
Please start proper Python Gym Agent
COMET INFO: ----------------------------
COMET INFO: Comet.ml Experiment Summary:
COMET INFO:   Data:
COMET INFO:     url: https://www.comet.com/sheila-janota/rl-in-wifi/d8358c0294004b81a21bd48d5e3027d7
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     Chosen CW [89985]           : (16.0, 1024.0)
COMET INFO:     Cumulative reward [15]      : (2506.361328125, 3116.0634765625)
COMET INFO:     Current throughput [89985]  : (23.913759231567383, 81.78258514404297)
COMET INFO:     Fairness index [89985]      : (0.03954099863767624, 0.9975709915161133)
COMET INFO:     Megabytes sent [89985]      : (0.2568970024585724, 2257.1972453296185)
CO

Sent 2251.00 Mb/s.	Mean speed: 37.52 Mb/s	Episode 15/15 finished



COMET INFO: Uploading stats to Comet before program termination (may take several seconds)


Training finished.


# Execution num4:

In [1]:
#%%
from ns3gym import ns3env
from comet_ml import Experiment, Optimizer
import tqdm
import subprocess
from collections import deque
import numpy as np

from agents.ddpg.agent import Agent, Config
from agents.ddpg.model import Actor
from agents.teacher import Teacher, EnvWrapper
from preprocessor import Preprocessor

In [2]:
#%%
scenario = "convergence"

simTime = 60 # seconds
stepTime = 0.01  # seconds
history_length = 300

EPISODE_COUNT = 15
steps_per_ep = int(simTime/stepTime)

sim_args = {
    "simTime": simTime,
    "envStepTime": stepTime,
    "historyLength": history_length,
    "agentType": Agent.TYPE,
    "scenario": "convergence",
    "nWifi": 30,
}

print("Steps per episode:", steps_per_ep)

threads_no = 1
env = EnvWrapper(threads_no, **sim_args)

#%%
env.reset()
ob_space = env.observation_space
ac_space = env.action_space

print("Observation space shape:", ob_space)
print("Action space shape:", ac_space)

assert ob_space is not None

Steps per episode: 6000
Waiting for simulation script to connect on port: tcp://localhost:33175
Please start proper ns-3 simulation script using ./waf --run "..."
Waf: Entering directory `/home/samuel/ns-allinone-3.29/ns-3.29/build'
Waf: Leaving directory `/home/samuel/ns-allinone-3.29/ns-3.29/build'
Build commands will be stored in build/compile_commands.json
'build' finished successfully (2.749s)


Ns3Env parameters:
--nWifi: 30
--simulationTime: 60
--openGymPort: 33175
--envStepTime: 0.01
--seed: -1
--agentType: continuous
--scenario: convergence
--dryRun: 0
Simulation started


Observation space shape: (1, 300)
Action space shape: (1, 1)


Simulation process id: 796653 (parent (waf shell) id: 796616)
Waiting for Python process to connect on port: tcp://localhost:33175
Please start proper Python Gym Agent


In [3]:
#%%
teacher = Teacher(env, 1, Preprocessor(False))

lr_actor = 4e-4
lr_critic = 4e-3

config = Config(buffer_size=4*steps_per_ep*threads_no, batch_size=32, gamma=0.7, tau=1e-3, lr_actor=lr_actor, lr_critic=lr_critic, update_every=1)
agent = Agent(history_length, action_size=1, config=config, actor_layers=[8, 128, 16], critic_layers=[8,128,16])

# Test the model
hyperparams = {**config.__dict__, **sim_args}
tags = ["Rew: normalized speed",
        f"{Agent.NAME}",
        sim_args['scenario'],
        f"Actor: {lr_actor}",
        f"Critic: {lr_critic}",
        f"Instances: {threads_no}",
        f"Station count: {sim_args['nWifi']}",
        *[f"{key}: {sim_args[key]}" for key in list(sim_args)[:3]]]
# agent.save()
logger = teacher.train(agent, EPISODE_COUNT,
                        simTime=simTime,
                        stepTime=stepTime,
                        history_length=history_length,
                        send_logs=True,
                        experimental=True,
                        tags=tags,
                        parameters=hyperparams)
# logger = teacher.eval(agent,
#                         simTime=simTime,
#                         stepTime=stepTime,
#                         history_length=history_length,
#                         tags=tags,
#                         parameters=hyperparams)
# agent.save()

CuDNN version: 7102
cpu


COMET INFO: Experiment is live on comet.ml https://www.comet.com/sheila-janota/rl-in-wifi/1fa9bc79946c4736af8d9e363e00e4a2

  0%|          | 0/6300 [00:00<?, ?it/s]

0


  5%|▌         | 333/6300 [02:33<1:03:54,  1.56it/s, curr_speed=34.66 Mbps, mb_sent=11.09 Mb]

------- STARTED TRAINING -------


...

Sent 2248.73 Mb/s.	Mean speed: 37.48 Mb/s	Episode 14/15 finished

14
Turning off noise


100%|██████████| 6300/6300 [48:31<00:00,  2.16it/s, curr_speed=37.56 Mbps, mb_sent=2246.90 Mb]


Waiting for simulation script to connect on port: tcp://localhost:37608
Please start proper ns-3 simulation script using ./waf --run "..."
Waf: Entering directory `/home/samuel/ns-allinone-3.29/ns-3.29/build'
Waf: Leaving directory `/home/samuel/ns-allinone-3.29/ns-3.29/build'
Build commands will be stored in build/compile_commands.json
'build' finished successfully (2.262s)


Ns3Env parameters:
--nWifi: 30
--simulationTime: 60
--openGymPort: 37608
--envStepTime: 0.01
--seed: -1
--agentType: continuous
--scenario: convergence
--dryRun: 0
Simulation started
Simulation process id: 1909239 (parent (waf shell) id: 1909210)
Waiting for Python process to connect on port: tcp://localhost:37608
Please start proper Python Gym Agent
COMET INFO: ----------------------------
COMET INFO: Comet.ml Experiment Summary:
COMET INFO:   Data:
COMET INFO:     url: https://www.comet.com/sheila-janota/rl-in-wifi/1fa9bc79946c4736af8d9e363e00e4a2
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     Chosen CW [89985]           : (16.0, 1024.0)
COMET INFO:     Cumulative reward [15]      : (2525.06640625, 3143.004150390625)
COMET INFO:     Current throughput [89985]  : (19.54650115966797, 54.808406829833984)
COMET INFO:     Fairness index [89985]      : (0.03962400183081627, 0.9974960088729858)
COMET INFO:     Megabytes sent [89985]      : (0.2122189998626709, 2254.5500786453485)

Sent 2246.90 Mb/s.	Mean speed: 37.45 Mb/s	Episode 15/15 finished

Training finished.
